In [1]:
from pyspark.mllib.clustering import KMeans
from numpy import array, random
from math import sqrt
from pyspark import SparkConf, SparkContext
from sklearn.preprocessing import scale

In [2]:
k =5 # no of clusters

In [3]:
conf = SparkConf().setMaster("local").setAppName("KMeansApp")
sc = SparkContext(conf=conf)


In [7]:
def createClusteredData(N, k):
    random.seed(10)
    pointsPerCluster = float(N)/k
    X = []
    for i in range (k):
        incomeCentroid = random.uniform(20000.0, 200000.0)
        ageCentroid = random.uniform(20.0, 70.0)
        for j in range(int(pointsPerCluster)):
            X.append([random.normal(incomeCentroid, 10000.0), random.normal(ageCentroid, 2.0)])
    X = array(X)
    return X

random.seed(0)


In [8]:
data = sc.parallelize(scale(createClusteredData(100, k)))

In [9]:
clusters = KMeans.train(data, k, maxIterations=10, initializationMode="random")